In [1]:
using Revise
using LinearAlgebra

In [2]:
# parameters
R = 6
N = 2^R
grid_min = -N/2 # minimum of the grid
grid_max = N/2 # maximum of the grid
xs = collect(Float64, grid_min:1:grid_max-1)
;

# (20)式:

$$
\left.\varphi_T(\vec{z})=\exp \left[i \sum_j^d z_j \mu_j-\frac{1}{2} \sum_j^d \sum_k^d \sigma_j \sigma_k z_j z_k \Sigma_{j k}\right)\right]
$$
where
$$
\mu_j=x_0^j+r_j T-\frac{1}{2} \sigma_j^2 \Sigma_{j j} T
$$

In [3]:
function phi_t_1d(j)
    η = 0.5
    T = 1.0
    r = 0.01
    σ = 0.2
    S0 = 100.0
    x0 = log(S0)
    d = length(j)
    α = 5.0 / d
    β = 0.5
    Σ_diag = 1.0
    Σ_non_diag = β / (1 + β)
    μ = x0 + r * T - 0.5 * σ^2 * Σ_diag * T

    phi1 = 0.0
    for i in 1:length(j)
        phi1 += ( - η * j[i] - im * α ) * μ
    end

    phi2_diag = 0.0
    for i in 1:length(j)
        phi2_diag += 0.5 * σ^2 * ( - η * j[i] - im * α )^2 * Σ_diag
    end

    phi2_non_diag = 0.0
    for i in 1:length(j), k in 1:length(j)
        if i != k
            phi2_non_diag += 0.5 * σ^2 * ( - η * j[i] - im * α ) * ( - η * j[k] - im * α ) * Σ_non_diag
        end
    end

    return exp( im * phi1 - (phi2_diag + phi2_non_diag) )
end

phi_t_1d(1.0)

-1.1481468691523573e10 - 1.056829417713916e10im

# (22)式:

$$
\hat{v}_{\min }(\vec{z})=-\frac{K^{1+i \sum_{j=1}^d z_j}}{(-1)^d\left(1+i \sum_{j=1}^d z_j\right) \prod_{j=1}^d (i z_j)}
$$

In [4]:
function v_min_1d(j)
    η = 0.5
    K = 100.0
    d = length(j)
    α = 5.0 / d

    op_seki = 1.0
    for i in 1:length(j)
        op_seki *= ( im * ( η * j[i] + im * α ) )
    end

    op_wa = 0.0
    for i in 1:length(j)
        op_wa += ( im * ( η * j[i] + im * α ) )
    end

    result = - ( K ^ ( 1 + op_wa ) ) / ( (-1)^d * ( 1 + op_wa) * op_seki )
    return result
end

v_min_1d(1.0)

-4.0322659783865716e-10 + 2.8482449009118854e-10im

# (25)式の内積の計算

$$
V\left(t, K \mid S_0\right)=\frac{e^{-r(T-t)}}{2 \pi}\left\langle\hat{v}_{\min } \mid \varphi_T\right\rangle
$$

In [5]:
inner_1d = 0.0
for x in xs
    inner_1d += phi_t_1d(x)*v_min_1d(x)
end
inner_1d

107.02763599855965 + 0.0014631648614676254im

In [6]:
T = 1 # 取引の日。1年後
r = 0.01 # 金利
t = 0.0 # 初期時刻
d = 1
η = 0.5

result = η * exp( - r * ( T - t ) ) * inner_1d / ( 2 * π )

8.432243205198837 + 0.00011527641291976327im

In [7]:
# モンテカルロと比較
monte_100 = 8.430171758105075
ratio = result/monte_100

1.0002457182550013 + 1.3674266222266742e-5im